In [2]:
#!pip install tensorflow librosa

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JorgeRuizDev/SpotMyFM/blob/main/Ludwig/notebooks/gtzan/gtzan_demo_transfer_learning.ipynb)
[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/JorgeRuizDev/SpotMyFM/blob/main/Ludwig/notebooks/gtzan/gtzan_demo_transfer_learning.ipynb)

In [3]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import os
from pathlib import Path
from IPython.core.display import HTML, display

import librosa
import random
import numpy as np
from math import floor
from typing import Union, Any

In [4]:
def download():
  #!pip install kaggle
  import os
  os.environ['KAGGLE_USERNAME'] = "jorgeruizdev"
  os.environ['KAGGLE_KEY'] = "5f3badd86b1d1035e2723792a67a2879"
  !kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
  !unzip -qq /content/gtzan-dataset-music-genre-classification.zip
#download()

In [5]:
print(tf. __version__)
print(tf.config.list_physical_devices('GPU'))


2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# CONFIG

SAMPLE_RATE = 22050

TRACK_LENGTH = 30

TRACK_SAMPLES = 22050 * TRACK_LENGTH

SPLITS = 10

SAMPLES_PER_SPLIT = TRACK_SAMPLES // SPLITS

N_MELS = 64

print(f"Samples Per Split {SAMPLES_PER_SPLIT}")

INPUT_SHAPE = (SAMPLES_PER_SPLIT , N_MELS, 1)

# Dataset:
DATASET_PATH = "/content/Data"

GENRES_PATH = f"{DATASET_PATH}/genres_original"

GENRES = os.listdir(GENRES_PATH)



# Training
BATCH_SIZE = 20


Samples Per Split 66150


In [7]:
GENRE_2_LABEL = {k: v for v, k in enumerate(GENRES)}
LABEL_2_GENRE = {v: k for k,v in GENRE_2_LABEL.items()}
print(f"Genre to Label Dict: {GENRE_2_LABEL}")
print(f"Label to Genre Dict: {LABEL_2_GENRE}")

Genre to Label Dict: {'country': 0, 'hiphop': 1, 'classical': 2, 'pop': 3, 'blues': 4, 'metal': 5, 'rock': 6, 'reggae': 7, 'disco': 8, 'jazz': 9}
Label to Genre Dict: {0: 'country', 1: 'hiphop', 2: 'classical', 3: 'pop', 4: 'blues', 5: 'metal', 6: 'rock', 7: 'reggae', 8: 'disco', 9: 'jazz'}


In [8]:
if not len(list(Path(f"{DATASET_PATH}/genres_original/disco").glob("*.wav"))) >= 100:
  print(f"""
    No subfolder 'genres_original/disco' found under {DATASET_PATH}.
    Please download the datasate from https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification and
    unzip it under {DATASET_PATH}.
  """)
  raise IOError


In [9]:
def split_track(y, samples_per_split = SAMPLES_PER_SPLIT):

    return [y[slice_samp:slice_samp + samples_per_split] for slice_samp in range(0, len(y), samples_per_split) ]



In [10]:
def normalize_mfccs(mfcc_feat: np.ndarray):
    return np.subtract(mfcc_feat,np.mean(mfcc_feat))


In [11]:
def preprocess_track(signal: Union[np.ndarray, Any], sr=SAMPLE_RATE):
    mfccs = librosa.feature.mfcc(y=signal, n_mfcc=N_MELS, sr=sr)

    #log_S = librosa.amplitude_to_db(mfccs)
    #delta = librosa.feature.delta(mfccs, order=1)

    return normalize_mfccs(mfccs)

In [12]:
def load_data(genre_path=GENRES_PATH):
    track_input = []
    labels = []
    for genre in GENRES:
        genre_folder = f"{genre_path}/{genre}"
        label = GENRE_2_LABEL[genre]
        for track in os.listdir(genre_folder):

            if (track == "jazz.00054.wav"):
                # This track is broken 
                continue

            y, sr = librosa.load(f"{genre_folder}/{track}", sr=SAMPLE_RATE)
            
            # Split into chunkso f SAMPLES_PER_SPLIT length 
            splits = split_track(y)

            track = []

            for split in splits:
                
                if (len(split) == SAMPLES_PER_SPLIT):
                    track.append(preprocess_track(split, sr))
            
            if len(track):  
                track_input.append(track)
                labels.append(label)

    

    return track_input, labels

In [13]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b), f"len(a) = {len(a)} != len(b) = {len(b)}"
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [14]:
tracks_, labels_ = load_data()

In [15]:
def train_test_val(ds, test=0.1, val=0.2):


    train_slice = floor(len(ds) * (1 - val + test))


    train = ds[:train_slice]
    rest = ds[train_slice:]

    rest_slice =  floor(len(rest) * (1 - (test / ( test + val))))

    test = rest[:rest_slice]
    val = rest[rest_slice:]

    return train, test, val


In [16]:
class TrackDataGen(keras.utils.Sequence):
    
    def __init__(self, data,
                 input_shape = (13, 259),
                 batch_size = 30):
        
        self.batch_size = batch_size
        self.shape = input_shape
        X = []
        Y = []

        for track_splits, labels in data:
            X.extend(track_splits)
            Y.extend(([labels]* len(track_splits)))

        X_np = np.array(X)
        X_np = np.expand_dims(X_np, axis=3)
        Y_np = np.array(Y)

        self.X, self.Y = unison_shuffled_copies(X_np, Y_np)
        
    
    
    def on_epoch_end(self):
        pass
    
    def __getitem__(self, idx):
        batch_x = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.Y[idx * self.batch_size:(idx + 1) * self.batch_size] 
        
        return batch_x, np.array(batch_y)
    
    def __len__(self):
        return len(self.X) // self.batch_size

In [22]:
class StopCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get("accuracy") or 0) > 0.97:
            self.model.stop_training = True
stop_callback = StopCallback()

In [21]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "./checkpoint",
    monitor='val_accuracy',
)

In [18]:
tracks, labels = tracks_, labels_


track_labels = list(zip(tracks, labels))

random.shuffle(track_labels)

train, test, val = train_test_val(track_labels)
print(len(train) + len(test) + len(val))


train_generator = TrackDataGen(train)
print(train_generator[0][0].shape)
validation_generator = TrackDataGen(val)




999
(30, 64, 130, 1)


In [23]:
base_model = keras.applications.ResNet50(
    include_top=False, 
    #input_shape=INPUT_SHAPE,
)

# Freeze the model
base_model.trainable = False

model = keras.models.Sequential([layers.Conv2D(3,(3,3),padding='same'), base_model, layers.Flatten(), layers.Dense(512, activation="relu"), layers.Dropout(0.2), layers.Flatten(), layers.Dense(512, activation="relu"),layers.Dense(10, activation='softmax')]) 


model.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [24]:
history = model.fit(train_generator,
                     validation_data = validation_generator, 
                     #validation_steps = len(tracks) // BATCH_SIZE,
                     epochs=1000,
                     callbacks = [stop_callback, checkpoint_callback]
                     )

299/299 [==============================] - 45s 139ms/step - loss: 2.3240 - accuracy: 0.4739 - val_loss: 1.4491 - val_accuracy: 0.4970
Epoch 2/1000
299/299 [==============================] - 41s 136ms/step - loss: 1.1865 - accuracy: 0.6103 - val_loss: 0.9459 - val_accuracy: 0.6818
Epoch 3/1000
299/299 [==============================] - 41s 136ms/step - loss: 0.9753 - accuracy: 0.6751 - val_loss: 1.0505 - val_accuracy: 0.6273
Epoch 4/1000
299/299 [==============================] - 41s 136ms/step - loss: 0.8369 - accuracy: 0.7237 - val_loss: 1.3077 - val_accuracy: 0.6485
Epoch 5/1000
299/299 [==============================] - 40s 135ms/step - loss: 0.7315 - accuracy: 0.7606 - val_loss: 0.9824 - val_accuracy: 0.6758
Epoch 6/1000
299/299 [==============================] - 41s 137ms/step - loss: 0.6464 - accuracy: 0.7855 - val_loss: 1.0187 - val_accuracy: 0.7333
Epoch 7/1000
299/299 [==============================] - 41s 136ms/step - loss: 0.5467 - accuracy: 0.8146 - val_loss: 1.1269 - val_a

In [25]:

test_generator = TrackDataGen(test)
results = model.evaluate(test_generator)
print("test loss, test acc:", results)

21/21 [==============================] - 1s 32ms/step - loss: 5.6727 - accuracy: 0.6333
test loss, test acc: [5.672662734985352, 0.6333333253860474]


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
model.save('/tmp/keras_model')

In [ ]:
import requests
from IPython import display
url = input()
r = requests.get(url, allow_redirects=True)
open('test.mp3', 'wb').write(r.content)

Y, sr = librosa.load('test.mp3')

display.Audio("test.mp3")

In [ ]:
splits = []

for split in split_track(Y):
    if len(split) == SAMPLES_PER_SPLIT:
        splits.append(preprocess_track(split))

print(np.array(splits).shape)
pred = model.predict(np.expand_dims(splits, axis=3))
#print(pred)
pred2 = pred.sum(axis=0)
print(pred2)
print(LABEL_2_GENRE)
print(LABEL_2_GENRE.get(np.argmax(pred2)))

